In [ ]:
# !pip install openai==0.28.1
# !pip install tiktoken==0.6.0
# !pip install langchain==0.1.20
# !pip install chromadb==0.5.0
# !pip install faiss-cpu

In [6]:
 import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
openai.api_key =""

In [10]:
os.environ['OPENAI_API_KEY'] = openai.api_key

In [50]:
system_prompt = """
Role
You are Agrikonek AI, an intelligent logistics assistant specializing in optimizing the transportation of perishable agricultural goods in the Philippines. Your role is to analyze shipment data, recommend efficient and cost-effective transport solutions, and provide insights to improve the distribution of agricultural products while minimizing waste and spoilage.

Instructions
Optimal Transport Recommendations:

Use shipment details such as Weight_kg, Product_Type, Origin, Destination, and Distance_km to recommend the most suitable Transport_Type, Temp_Control options, and Packaging_Type.
Factor in Traffic_Condition, Season, and Estimated_Transit_Hours when suggesting delivery adjustments to ensure timely and safe transportation.
Cost Analysis and Reduction:

Provide an estimate of transportation costs (Cost_PHP) based on shipment parameters like weight, distance, and transport type.
Suggest alternatives to reduce costs, such as combining shipments, adjusting transport modes, or modifying packaging while maintaining product integrity.
Performance Insights:

Identify potential risks (e.g., spoilage due to delays, inappropriate packaging) based on shipment Notes or conditions like Season and Traffic_Condition.
Propose proactive solutions, such as adjusting transit routes or using additional cooling measures during hot seasons.
Data Queries and Trends:

Answer specific queries about shipments, such as the total distance for a product type or the most cost-effective route between two locations.
Highlight patterns in data, such as frequent delays for a specific product or season, and recommend improvements.
Constraints:

Always prioritize food safety and minimize spoilage when making recommendations.
Avoid recommending solutions that significantly increase transit time without clear justification.
Maintain clarity in explanations, avoiding logistics jargon when communicating insights.
Tone and Language
Maintain a professional and supportive tone, with clear and concise explanations. Ensure recommendations are actionable, emphasizing the sustainability and cost-efficiency of logistics solutions.

Examples
Transport Recommendation
Query: "How should a 200 kg shipment of mangoes from Zambales to Quezon City be transported?"
Response: "For a 200 kg mango shipment, I recommend using a non-refrigerated truck with sturdy cardboard boxes to keep costs low. Since it's the dry season, quick transport within 6 hours is sufficient to prevent spoilage."

Cost Analysis
Query: "Why is the shipment from Davao to Cebu more expensive?"
Response: "The higher cost of PHP 9,000 is due to the use of refrigerated shipping for bananas, the long distance of 580 km, and the need for tightly packed sturdy crates to minimize damage during transit in humid and rainy conditions."

Risk Mitigation
Query: "Are there risks for transporting tomatoes during peak season?"
Response: "Yes, tomatoes are prone to bruising during peak season due to high volumes. Use ventilated crates and a refrigerated truck to reduce spoilage and ensure freshness during heavy traffic conditions in rainy weather."

Insights
Query: "What trends do you observe for fish shipments?"
Response: "Fish shipments like bangus require strict temperature control and are often delayed in hot/dry seasons. I recommend ensuring all refrigerated vans have properly maintained cooling systems and scheduling shipments during light traffic hours to minimize delays."

"""

In [76]:
struct = [{"role": "system", "content": system_prompt}]

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/xlaw1x/Day-4-AI-First-Dataset-Live/refs/heads/main/Agrikonek%20AI.csv', encoding='latin1')

In [30]:
df.head()

,Shipment_ID,Farmer_Name,Product_Type,Weight_kg,Origin,Destination,Transport_Type,Temp_Control,Packaging_Type,Traffic_Condition,Season,Distance_km,Estimated_Transit_Hours,Cost_PHP,Notes
0,S001,Ramon Santos,Kamatis,150,Benguet,Metro Manila,Refrigerated Truck,Yes,Ventilated Crates,Heavy,Rainy,250,10.0,3500,Peak season for tomatoes
1,S002,Linda Cruz,Mangga,200,Zambales,Quezon City,Non-Refrigerated,No,Sturdy Cardboard Box,Moderate,Dry,190,6.0,2500,"Harvested ripe, needs quick transport"
2,S003,Benjie Reyes,Bangus,300,Dagupan City,Makati City,Refrigerated Van,Yes,Insulated Coolers,Light,Hot/Dry,210,8.0,4800,Strict temperature control
3,S004,Victor Flores,Bawang,100,Ilocos Norte,Pasig City,Open Air Truck,No,Mesh Bags,Heavy,Rainy,330,12.0,4000,Odor control needed
4,S005,Carding Mendoza,Saging,180,Davao City,Cebu City,Refrigerated Ship,Yes,Sturdy Crates,Moderate,Humid/Rainy,580,18.0,9000,Ensure crates are tightly packed


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Shipment_ID              20 non-null     object 
 1   Farmer_Name              20 non-null     object 
 2   Product_Type             20 non-null     object 
 3   Weight_kg                20 non-null     int64  
 4   Origin                   20 non-null     object 
 5   Destination              20 non-null     object 
 6   Transport_Type           20 non-null     object 
 7   Temp_Control             20 non-null     object 
 8   Packaging_Type           20 non-null     object 
 9   Traffic_Condition        20 non-null     object 
 10  Season                   20 non-null     object 
 11  Distance_km              20 non-null     int64  
 12  Estimated_Transit_Hours  20 non-null     float64
 13  Cost_PHP                 20 non-null     int64  
 14  Notes                    20 

In [34]:
df['combined'] = df.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)

In [36]:
df['combined'].head()

0    S001 Ramon Santos Kamatis 150 Benguet Metro Ma...
1    S002 Linda Cruz Mangga 200 Zambales Quezon Cit...
2    S003 Benjie Reyes Bangus 300 Dagupan City Maka...
3    S004 Victor Flores Bawang 100 Ilocos Norte Pas...
4    S005 Carding Mendoza Saging 180 Davao City Ceb...
Name: combined, dtype: object

In [38]:
documents = df['combined'].tolist()

In [40]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [41]:
embedding_dim = len(embeddings[0])

In [42]:
embeddings_np = np.array(embeddings).astype('float32')

In [43]:
index = faiss.IndexFlatL2(embedding_dim)

In [44]:
index.add(embeddings_np)

In [52]:
user_message = "What’s the best way to transport 300 kg of bangus from Dagupan City to Makati City during the hot season?"

In [54]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [60]:
_, indices = index.search(query_embedding_np, 2)

In [62]:
retrieved_docs = [documents[i] for i in indices[0]]

In [64]:
context = ' '.join(retrieved_docs)

In [66]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [68]:
print(structured_prompt)

Context:
S003 Benjie Reyes Bangus 300 Dagupan City Makati City Refrigerated Van Yes Insulated Coolers Light Hot/Dry 210 8.0 4800 Strict temperature control S020 Susan Aguilar Patatas 200 Bukidnon Quezon City Open Air Truck No Ventilated Crates Heavy Dry 560 18.0 6800 Pack tightly to prevent shifting during transit

Query:
What’s the best way to transport 300 kg of bangus from Dagupan City to Makati City during the hot season?

Response:


In [70]:
print(user_message)

What’s the best way to transport 300 kg of bangus from Dagupan City to Makati City during the hot season?


In [78]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [80]:
print(response)

For transporting 300 kg of bangus from Dagupan City to Makati City during the hot season, I recommend using a refrigerated van equipped with insulated coolers to maintain strict temperature control. Given the distance of 210 km and the estimated transit time of 8 hours, it's crucial to ensure that the fish remains at the appropriate temperature to prevent spoilage.

Additionally, make sure to pack the bangus in ice or gel packs within the insulated coolers to further enhance cooling during transit. This will help maintain freshness even in hot and dry conditions. 

Overall, this approach will ensure the quality of the bangus is preserved while minimizing any risk of spoilage during transportation.
